#### Part 1: Create a framework for the game mechanics/rules ####

In [ ]:
import numpy as np
import os
import sys
import time

def othello_game(board_size=4):

    if not (isinstance(board_size, int) and board_size > 0 and board_size % 2 == 0):
        raise ValueError("board_size must be an even positive integer.")

    board = np.full((board_size, board_size), '.', dtype=str)
    board[board_size//2-1][board_size//2-1] = board[board_size//2][board_size//2] = 'B' 
    board[board_size//2-1][board_size//2] = board[board_size//2][board_size//2-1] = 'W'

    def clear_screen():
        try:
            from IPython.display import clear_output
            clear_output(wait=True)
        except Exception:
            pass
    
    def print_board(board, cell_size=1):
        n = len(board)
        if any(len(row) != n for row in board):
            raise ValueError("Board must be square")
        if cell_size < 1:
            raise ValueError("cell_size must be >= 1")

        GREEN = "\033[32m"
        BLUE = "\033[34m"
        BLACK = "\033[30m"
        WHITE = "\033[97m"
        RESET = "\033[0m"
        GREEN_BG = "\033[42m"   

        TL, TM, TR = GREEN + '┌' + RESET, GREEN + '┬' + RESET, GREEN + '┐' + RESET
        ML, MM, MR = GREEN + '├' + RESET, GREEN + '┼' + RESET, GREEN + '┤' + RESET
        BL, BM, BR = GREEN + '└' + RESET, GREEN + '┴' + RESET, GREEN + '┘' + RESET
        HL = GREEN + '─' + RESET
        VT = GREEN + '│' + RESET

        label_w = max(2, len(str(n)))
        cell_inner = cell_size + 4       
        cell_h = cell_inner - 4          

        hor = HL * cell_inner

        top_border = TL + (hor + TM) * (n - 1) + hor + TR
        mid_border = ML + (hor + MM) * (n - 1) + hor + MR
        bot_border = BL + (hor + BM) * (n - 1) + hor + BR

        cols = [chr(ord('A') + i) for i in range(n)]
        header = ' ' * (label_w + 1)
        header += ' '.join(BLUE + col.center(cell_inner) + RESET for col in cols)
        print('')
        print(' ' + header)

        print(' ' * (label_w + 1) + top_border)

        mid_line_index = cell_h // 2
        for r in range(n):
            for inner_row in range(cell_h):
                if inner_row == mid_line_index:
                    row_label = BLUE + str(r + 1).rjust(label_w) + ' ' + RESET
                else:
                    row_label = ' ' * (label_w + 1)

                row_cells = []
                for c in range(n):
                    val = board[r][c]
                    if inner_row == mid_line_index:
                        if val == 'B':
                            ch = GREEN_BG + BLACK + ' ● ' + RESET
                        elif val == 'W':
                            ch = GREEN_BG + WHITE + ' ● ' + RESET
                        else:
                            ch = GREEN_BG + '   ' + RESET
                        cell_text = ' ' + ch.center(cell_inner) + ' '
                    else:
                        cell_text = GREEN_BG + '   ' * (cell_inner +2) + RESET
                    row_cells.append(cell_text)

                print(row_label + VT + VT.join(row_cells) + VT)

            if r < n - 1:
                print(' ' * (label_w + 1) + mid_border)
            else:
                print(' ' * (label_w + 1) + bot_border)

    def is_valid_move(row, col, player):
        if board[row][col] != '.':
            return False
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = row + dx, col + dy
            found_opponent = False
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    found_opponent = True
                elif board[nx][ny] == player:
                    if found_opponent:
                        return True
                    else:
                        break
                else:
                    break
                nx += dx
                ny += dy
        return False

    def make_move(row, col, player):
        board[row][col] = player
        opponent = 'W' if player == 'B' else 'B'
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dx, dy in directions:
            nx, ny = row + dx, col + dy
            to_flip = []
            while 0 <= nx < board_size and 0 <= ny < board_size:
                if board[nx][ny] == opponent:
                    to_flip.append((nx, ny))
                elif board[nx][ny] == player:
                    for fx, fy in to_flip:
                        board[fx][fy] = player
                    break
                else:
                    break
                nx += dx
                ny += dy

    def has_valid_moves(player):
        for i in range(board_size):
            for j in range(board_size):
                if is_valid_move(i, j, player):
                    return True
        return False

    current_player = 'B'
    passes = 0  # Count consecutive passes
    
    while True:
        clear_screen()
        b_count = np.sum(board == 'B')
        w_count = np.sum(board == 'W')
        
        # Check game end conditions
        if (b_count + w_count == board_size * board_size or 
            b_count == 0 or w_count == 0 or passes >= 2):
            print_board(board)
            print("\nGame over!")
            if b_count > w_count:
                print(f"\nBlack wins {b_count}-{w_count}!")
            elif w_count > b_count:
                print(f"\nWhite wins {w_count}-{b_count}!")
            else:
                print(f"\nIt's a tie {b_count}-{w_count}!")
            break

        print_board(board)
        print(f"\nCurrent player: {current_player}")
        print(f"\nCurrent counts: B = {b_count},  W = {w_count}.")

        # Check if current player has any valid moves
        if not has_valid_moves(current_player):
            print(f"\nNo valid moves for player {current_player}. Passing turn.")
            current_player = 'W' if current_player == 'B' else 'B'
            passes += 1
            time.sleep(2)
            continue
        else:
            passes = 0  # Reset passes counter when a valid move is available
        
        move = input("Enter your move coordinate (e.g., A1) or 'end' to quit: ").strip()
        if not move:
            print("\nInvalid input. Please enter a move.")
            time.sleep(2)
            continue
            
        if move.lower() == 'end':
            print("\nGame ended.")
            if b_count > w_count:
                print(f"\nBlack wins {b_count}-{w_count}!")
            elif w_count > b_count:
                print(f"\nWhite wins {w_count}-{b_count}!")
            else:
                print(f"\nIt's a tie {b_count}-{w_count}!")
            break

        try:
            if len(move) < 2:
                raise ValueError("\nInvalid input format. Use format like 'A1'")
                
            row_letter = move[0].upper()
            col_str = move[1:]  # Support multi-digit numbers
            
            if row_letter < 'A' or row_letter > chr(ord('A') + board_size - 1):
                raise ValueError(f"\nRow letter must be between A and {chr(ord('A') + board_size - 1)}")
                
            col = int(col_str)
            if col < 1 or col > board_size:
                raise ValueError(f"\nColumn number must be between 1 and {board_size}")
                
            row = int(col_str) - 1
            col = ord(row_letter) - ord('A')
            
            if is_valid_move(row, col, current_player):
                make_move(row, col, current_player)
                current_player = 'W' if current_player == 'B' else 'B'
            else:
                print("\nInvalid move. Try again.")
                time.sleep(2)
        except (ValueError, IndexError) as e:
            print(str(e) if str(e).startswith('\n') else "\nInvalid input. Please use format like 'A1'.")
            time.sleep(2)

othello_game(8)


      A     B     C     D     E     F     G     H  
   ┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
 1 │     │     │     │     │     │     │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 2 │     │     │  ●  │     │     │     │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 3 │     │     │     │  ●  │     │     │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 4 │     │     │     │  ●  │  ●  │  ●  │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 5 │     │     │     │  ●  │  ●  │     │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 6 │     │     │     │     │     │     │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 7 │     │     │     │     │     │     │     │     │
   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
 8 │     │     │     │     │     │     │     │     │
   └─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘

Current player: W

Current counts: B = 5,  W 